# Bitmask to polygonal annotation
This notebook contains functions that allow us to transform bitmasks to polygonal annotations in COCO format
## Main code

In [1]:
from PIL import Image # (pip install Pillow)
import matplotlib.pyplot as plt
import numpy as np                                 # (pip install numpy)
from skimage import measure                        # (pip install scikit-image)
from shapely.geometry import Polygon, MultiPolygon # (pip install Shapely)
import cv2
import os

In [2]:

def create_submask(my_img):
    sub_masks = {}
    width, height, _ = my_img.shape
    black = np.ones(3)*255

    for x in range(width):
        for y in range(height):
            
            color = my_img[x][y][:3]
            if((black==color).all()):
                pixel_str = str(color)
                sub_mask = sub_masks.get(pixel_str)
                if sub_mask is None:
                # Create a sub-mask (one bit per pixel) and add to the dictionary
                    # Note: we add 1 pixel of padding in each direction
                    # because the contours module doesn't handle cases
                    # where pixels bleed to the edge of the image
                    sub_masks[pixel_str] = np.zeros((width+2, height+2)) 
                # Set the pixel value to 1 (default is 0), accounting for padding
                sub_masks[pixel_str][x+1][y+1]=255
    return sub_masks



In [3]:
def create_submask_annotation(sub_mask, image_id, category_id, annotation_id, is_crowd):
    # Find contours (boundary lines) around each sub-mask
    # Note: there could be multiple contours if the object
    # is partially occluded. (E.g. an elephant behind a tree)
    contours = measure.find_contours(sub_mask, 0.5, positive_orientation='low')

    segmentations = []
    polygons = []
    for contour in contours:
        # Flip from (row, col) representation to (x, y)
        # and subtract the padding pixel
        for i in range(len(contour)):
            row, col = contour[i]
            contour[i] = (col - 1, row - 1)

        # Make a polygon and simplify it
        poly = Polygon(contour)
        poly = poly.simplify(1.0, preserve_topology=False)
        polygons.append(poly)
        segmentation = np.array(poly.exterior.coords).ravel().tolist()
        segmentations.append(segmentation)

    # Combine the polygons to calculate the bounding box and area
    multi_poly = MultiPolygon(polygons)
    x, y, max_x, max_y = multi_poly.bounds
    width = max_x - x
    height = max_y - y
    bbox = (x, y, width, height)
    area = multi_poly.area

    annotation = {
        'segmentation': segmentations,
        'iscrowd': is_crowd,
        'image_id': image_id,
        'category_id': category_id,
        'id': annotation_id,
        'bbox': bbox,
        'area': area
    }

    return annotation

In [5]:
def images_to_annotations(dir_path):
    #Initializing images and annotations lists
    images=[]
    annotations = []
    img_names = os.listdir(dir_path)
    img_license = 0
    img_id=0
    ann_id=0
    is_crowd=0
            
    for img_name in img_names:
        img = cv2.imread(dir_path + img_name)
        img_size = img.shape
        submasks = create_submask(img)
        for color, submask in submasks.items():
            img_cat = color2cat[color]
            annotation = create_submask_annotation(submask , img_id, img_cat, ann_id, is_crowd)
            annotations.append(annotation)
            ann_id+=1
            

        new_img={}
        new_img["license"] = img_license
        new_img["file_name"] = img_name.split(".")[0]+".jpg" #Changed to match the video images
        new_img["width"] = img_size[1]
        new_img["height"] = img_size[0]
        new_img["id"] = img_id
        images.append(new_img)

        # sub_masks = create_sub_masks()
        # for color, sub_mask in sub_masks.items():
        #     plt.imshow(sub_mask)
        #     plt.show()
        
        img_id+=1

    return annotations,images





In [6]:
my_dict = {}
color2cat={"[255 255 255]": 1}
info = {
      "description":"Test Dataset",
      "url":"",
      "version":"0.1",
      "year":2020,
      "contributor":"Josmar Suarez",
      "date_created":"2020/07/14"
   }

licenses = [{
        "url": "",
        "id": 0,
        "name": "Attribution-NonCommercial-ShareAlike License"
    }]
categories = [
      {
         "supercategory":"person",
         "id":1,
         "name":"person"
      }
   ]

In [7]:
dir_path = "../datasets/casia/DatasetB-1/silhouettes/001/bg-01/090/"
anns,imgs = images_to_annotations(dir_path)

my_dict["info"]= info
my_dict["licenses"]= licenses
my_dict["images"]=imgs
my_dict["categories"]=categories
my_dict["annotations"]=anns
print(my_dict.keys())

dict_keys(['info', 'licenses', 'images', 'categories', 'annotations'])


In [8]:
import json

with open('casia.json', 'w') as fp:
    json.dump(my_dict, fp)